In [ ]:
!pip install keras

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install kaggle

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!mkdir ~/.kaggle

In [ ]:
!cp kaggle.json ~/.kaggle/

In [ ]:
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d balabaskar/wonders-of-the-world-image-classification

100% 432M/432M [00:23<00:00, 16.8MB/s]
100% 432M/432M [00:23<00:00, 18.9MB/s]


In [ ]:
import zipfile
zip_ref = zipfile.ZipFile("/content/wonders-of-the-world-image-classification.zip", "r")
zip_ref.extractall('/content')
zip_ref.close()

In [ ]:
import os
#counting the number of files in train folder
path, dirs, files = next(os.walk('/content/Wonders of World/Wonders of World/burj_khalifa'))
f_count = len(files)
print("The number of images is: ", f_count)

The number of images is:  390


In [ ]:
import os
for dirname, _, filenames in os.walk('/content'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/content/wonders_of_world_images.csv
/content/wonders-of-the-world-image-classification.zip
/content/kaggle.json
/content/.config/.last_update_check.json
/content/.config/active_config
/content/.config/config_sentinel
/content/.config/.last_survey_prompt.yaml
/content/.config/gce
/content/.config/.last_opt_in_prompt.yaml
/content/.config/configurations/config_default
/content/.config/logs/2023.05.17/20.40.20.444857.log
/content/.config/logs/2023.05.17/20.41.21.257288.log
/content/.config/logs/2023.05.17/20.40.54.575736.log
/content/.config/logs/2023.05.17/20.41.22.007826.log
/content/.config/logs/2023.05.17/20.40.46.374560.log
/content/.config/logs/2023.05.17/20.39.53.943226.log
/content/Wonders of World/Wonders of World/machu_pichu/a1568cfa83.jpg
/content/Wonders of World/Wonders of World/machu_pichu/0847dd324c.jpg
/content/Wonders of World/Wonders of World/machu_pichu/3d8e7cd05a.jpg
/content/Wonders of World/Wonders of World/machu_pichu/224100bb1c.jpg
/content/Wonders of World/Wonder

In [ ]:
import tensorflow as tf
import keras

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

In [ ]:
datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2 # Set the validation split to 20%
)


In [ ]:
directory = '/content/Wonders of World/Wonders of World'

In [ ]:
train_ds = datagen.flow_from_directory(
    directory,
    target_size=(256, 256),
    batch_size=32,
    class_mode='categorical',
    subset='training' # Set the subset to 'training' for the training set
)

val_ds = datagen.flow_from_directory(
    directory,
    target_size=(256, 256),
    batch_size=32,
    class_mode='categorical',
    subset='validation' # Set the subset to 'validation' for the validation set
)

Found 3082 images belonging to 12 classes.
Found 764 images belonging to 12 classes.


In [ ]:
class_labels = train_ds.class_indices

In [ ]:
class_labels

{'burj_khalifa': 0,
 'chichen_itza': 1,
 'christ_the_reedemer': 2,
 'eiffel_tower': 3,
 'great_wall_of_china': 4,
 'machu_pichu': 5,
 'pyramids_of_giza': 6,
 'roman_colosseum': 7,
 'statue_of_liberty': 8,
 'stonehenge': 9,
 'taj_mahal': 10,
 'venezuela_angel_falls': 11}

In [ ]:
import pickle

In [ ]:
with open('/content/data.pkl', 'wb') as file:
    pickle.dump(class_labels, file)

In [ ]:
from keras import Sequential
from keras.layers import Dense,Conv2D,MaxPooling2D,Flatten,BatchNormalization,Dropout

In [ ]:
model = Sequential()

model.add(Conv2D(6, kernel_size=(5,5), activation = 'relu', input_shape=(256,256,3)))
model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2)))

model.add(Conv2D(16,kernel_size=(5,5),activation='relu'))

model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2)))

model.add(Conv2D(120,kernel_size=(5,5),activation='relu'))

model.add(Flatten())

model.add(Dense(84, activation='relu'))
model.add(Dense(12, activation='softmax'))

In [ ]:
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

In [ ]:
history = model.fit(train_ds, epochs = 10, validation_data = val_ds)

Epoch 1/10
97/97 [==============================] - 63s 501ms/step - loss: 3.2242 - accuracy: 0.2505 - val_loss: 1.8237 - val_accuracy: 0.3835
Epoch 2/10
97/97 [==============================] - 40s 418ms/step - loss: 1.3951 - accuracy: 0.5620 - val_loss: 1.3054 - val_accuracy: 0.5825
Epoch 3/10
97/97 [==============================] - 43s 443ms/step - loss: 0.7200 - accuracy: 0.7787 - val_loss: 1.0652 - val_accuracy: 0.6911
Epoch 4/10
97/97 [==============================] - 39s 404ms/step - loss: 0.2431 - accuracy: 0.9322 - val_loss: 1.0219 - val_accuracy: 0.7552
Epoch 5/10
97/97 [==============================] - 39s 405ms/step - loss: 0.1077 - accuracy: 0.9721 - val_loss: 1.2068 - val_accuracy: 0.7605
Epoch 6/10
97/97 [==============================] - 42s 435ms/step - loss: 0.0314 - accuracy: 0.9951 - val_loss: 1.3828 - val_accuracy: 0.7801
Epoch 7/10
97/97 [==============================] - 39s 404ms/step - loss: 0.0148 - accuracy: 0.9958 - val_loss: 1.5233 - val_accuracy: 0.7487

In [ ]:
model.save('/content/model.h5')